In [1]:
import torch
import argparse
import time
import math
import os, sys
import itertools
import pickle
from tqdm import tqdm
import numpy as np
import os.path as osp

In [2]:
torch.cuda.device_count()

1

In [3]:
def loadpkl(path):
    with open(path, 'rb') as f:
        rst=pickle.load(f)
    return rst

In [4]:
path="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml/full_id_abs_title_cite_ref_author.pkl"
full_data=loadpkl(path)

In [5]:
path="/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/f1_result/eval_test_gold.pkl"
gold_set=loadpkl(path)

In [6]:
path="/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/f1_result/eval_test_pred_1000.pkl"
pred_set=loadpkl(path)

In [7]:
candidate_set={}
candidate_ref_set={}
for k,v in pred_set.items():
    cur=set()
    for paper in v[:200]:
        cur.add(paper)
        refset=set()
        for ref in full_data[paper]["references"]:
            cur.add(ref)
            refset.add(ref)
        candidate_ref_set[paper]=refset
    candidate_set[k]=list(cur)

In [8]:
maxlen=0
for k,v in candidate_set.items():
    if len(v)>maxlen:
        maxlen=len(v)
print(maxlen)

9359


In [9]:
def load_network(model,path,device):
    loaded_net = torch.load(
        path,
        map_location=torch.device(self.device),
    )
    model.load_state_dict(loaded_net, strict=True)
    return model

In [12]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer,AutoModel
from torch import nn

In [13]:
class Mymodel(nn.Module):
    #self supervised learning sbert
    def __init__(self, model_name):
        super().__init__()
        self.cross_encoder =  AutoModelForSequenceClassification.from_pretrained(model_name)
        self.cross_encoder.gradient_checkpointing_enable()
       
    def forward(self, input,mode=None):
        output=self.cross_encoder(**input)
        return output

In [ ]:
model = Mymodel(cfg)